In [11]:
#Importing necessary modules
import boto3
import os
from boto3.session import Session
import pandas as pd
from time import sleep
import sagemaker
import s3fs
from sklearn.model_selection import train_test_split
import numpy as np
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
import xgboost as xgb
import json
from slugify import slugify
import csv

In [2]:
#Build the s3 bucket
data_bucket = 'tweet-analysis1'
subfolder = 'data'
dataset = 'inbound.csv'
session = Session(aws_access_key_id='****xxxxxxxxxxxxx',aws_secret_access_key='y**************yyyyy************')

boto = session.resource('s3')
boto.create_bucket(
    Bucket=data_bucket,
    CreateBucketConfiguration={'LocationConstraint':'eu-west-2'})
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-2'

In [4]:
#create a sub-folder inside the bucket
boto = boto3.client('s3')
directory_name = "data" #it's name of your folders
boto.put_object(Bucket=data_bucket, Key=(directory_name+'/'))

{'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Tue, 23 Mar 2021 16:58:37 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'cckCi8hboYKW6i+lSLTu3uiLoAQ9Bn5BNRqN9h0g3A0/LXQkX7+UA2g8eYxa2jNsBNOBUVgBYxU=',
   'x-amz-request-id': 'HJT4VNSHCG55SSBH'},
  'HTTPStatusCode': 200,
  'HostId': 'cckCi8hboYKW6i+lSLTu3uiLoAQ9Bn5BNRqN9h0g3A0/LXQkX7+UA2g8eYxa2jNsBNOBUVgBYxU=',
  'RequestId': 'HJT4VNSHCG55SSBH',
  'RetryAttempts': 0}}

In [5]:
#Upload the data from local into the created bucket
def upload_files(file_name, bucket,object_name=None, args=None):
    """
    file_name: Name of file on local
    bucket: bucket name
    object_name: name of file on s3
    args: custom args
    """
    s3=boto3.resource('s3')
    if object_name is None:
        object_name=file_name
    response=s3.meta.client.upload_file(file_name, bucket,object_name, ExtraArgs=args)  
    print(response)
upload_files('C:/Users/ds8/Boto3/data/inbound.csv',data_bucket,'data/inbound.csv')    

None


In [7]:
# Create a bucket policy
bucket_policy = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'AddPerm',
        'Effect': 'Allow',
        'Principal': '*',
        'Action': ['s3:GetObject'],
        'Resource': f'arn:aws:s3:::{data_bucket}/*'
    }]
}

# Convert the policy from JSON dict to string
bucket_policy = json.dumps(bucket_policy)

# Set the new policy
boto.put_bucket_policy(Bucket=data_bucket, Policy=bucket_policy)

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Tue, 23 Mar 2021 17:02:44 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'zOCYHpVAuW56p4i+wHB1kV+z8NDv1YDgsQs2y3FT1ViSontDPWSmn0oPdioddqVsPr2p52sM/94=',
   'x-amz-request-id': '3VTYSGE12D5GM9TV'},
  'HTTPStatusCode': 204,
  'HostId': 'zOCYHpVAuW56p4i+wHB1kV+z8NDv1YDgsQs2y3FT1ViSontDPWSmn0oPdioddqVsPr2p52sM/94=',
  'RequestId': '3VTYSGE12D5GM9TV',
  'RetryAttempts': 0}}

In [8]:
#read the data from the s3 bucket
%%time
df = pd.read_csv(f's3://{data_bucket}/{subfolder}/{dataset}')
display(df.head())

,tweet_id,author_id,created_at,in_reply_to,text,escalate
0,2,115712,Tue Oct 31 22:11:45 +0000 2017,sprintcare,@sprintcare and how do you propose we do that,False
1,3,115712,Tue Oct 31 22:08:27 +0000 2017,sprintcare,@sprintcare I have sent several private messag...,True
2,5,115712,Tue Oct 31 21:49:35 +0000 2017,sprintcare,@sprintcare I did.,False
3,16,115713,Tue Oct 31 20:00:43 +0000 2017,sprintcare,@sprintcare Since I signed up with you....Sinc...,False
4,22,115716,Tue Oct 31 22:16:48 +0000 2017,Ask_Spectrum,@Ask_Spectrum Would you like me to email you a...,False


Wall time: 25.4 s


In [9]:
print(f'Number of rows in dataset: {df.shape[0]}')
print(df['escalate'].value_counts())

Number of rows in dataset: 520793
False    417800
True     102993
Name: escalate, dtype: int64


In [10]:
# split the data into train and test
train_df, val_df, _, _ = train_test_split(df, df['escalate'], test_size=0.2, random_state=0)
print(f'{train_df.shape[0]} rows in training data')
print(f'{val_df.shape[0]} rows in validation data')

416634 rows in training data
104159 rows in validation data


In [14]:
%%time
def preprocess(df):
    all_rows = df.values.tolist()
    transformed_rows = list(map(transform_instance, all_rows))
    transformed_df = pd.DataFrame(transformed_rows)
    return transformed_df

def transform_instance(row):
    text = slugify(row[4], separator=' ')
    cur_row = []
    label = "__label__1" if row[5] == True else "__label__0" # Prefix 0 or 1 from sentiment
    cur_row.append(label)
    cur_row.append(text)
    return ' '.join(cur_row)

transformed_validation_rows = preprocess(val_df)
display(transformed_validation_rows.head())

,0
0,__label__1 115990 no joke this is one of the w...
1,__label__0 amazonhelp primeira camada https t ...
2,__label__1 microsofthelps my mistake
3,__label__1 770932 americanair they notoriously...
4,__label__1 amazonhelp neither man seems to kno...


Wall time: 1.89 s


In [16]:
#save data
s3 = s3fs.S3FileSystem(anon=False)
s3_validation_data = f's3://{data_bucket}/{subfolder}/processed/validation.csv'

data = transformed_validation_rows.to_csv(
        header=False, index=False, quoting=csv.QUOTE_NONE, sep='|', escapechar='^').encode()
with s3.open(s3_validation_data, 'wb') as f:
    f.write(data)
s3_train_data = f's3://{data_bucket}/{subfolder}/processed/train.csv'
transformed_train_rows = preprocess(train_df)
display(transformed_train_rows.head())
data = transformed_train_rows.to_csv(
        header=False, index=False, quoting=csv.QUOTE_NONE, sep='|', escapechar='^').encode()
with s3.open(s3_train_data, 'wb') as f:
    f.write(data)    

,0
0,__label__0 amazonhelp et en plus se faire engu...
1,__label__1 morrisons 641226 standard reply mor...
2,__label__1 idea cares 1936 116590 this is gr8 ...
3,__label__0 askamex yes i did weeks ago and no ...
4,__label__0 amazonhelp i dont want your stupid ...


In [18]:
%%time

#upload trai and validation data into the s3 bucket
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Wall time: 998 µs


In [19]:
#train the model
role = "arn:aws:iam::408997494357:role/service-role/AmazonSageMaker-ExecutionRole-20210217T114476"
s3_output_location = f's3://{data_bucket}/{subfolder}/output'

sess = sagemaker.Session()

container = sagemaker.amazon.amazon_estimator.get_image_uri(boto3.Session().region_name, "blazingtext", "latest")

estimator = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_max_run = 600,
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

estimator.set_hyperparameters(mode="supervised",
                            epochs=10,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

estimator.fit({'train': train_data, 'validation': validation_data})

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-03-23 17:17:58 Starting - Starting the training job...
2021-03-23 17:18:00 Starting - Launching requested ML instancesProfilerReport-1616519877: InProgress
......
2021-03-23 17:19:06 Starting - Preparing the instances for training...
2021-03-23 17:19:57 Downloading - Downloading input data
2021-03-23 17:19:57 Training - Downloading the training image..Arguments: train
[03/23/2021 17:20:11 WARNING 140467125986688] Loggers have already been setup.
[03/23/2021 17:20:11 WARNING 140467125986688] Loggers have already been setup.
[03/23/2021 17:20:11 INFO 140467125986688] nvidia-smi took: 0.025149822235107422 secs to identify 0 gpus
[03/23/2021 17:20:11 INFO 140467125986688] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[03/23/2021 17:20:11 INFO 140467125986688] Processing /opt/ml/input/data/train/train.csv . File size: 40.62648296356201 MB
[03/23/2021 17:20:11 INFO 140467125986688] Processing /opt/ml/input/data/valid

In [21]:
# deploy the model
endpoint_name = 'customer-support'
try:
    sess.delete_endpoint(sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name).endpoint)
    print('Warning: Existing endpoint deleted to make way for your new endpoint.')
    sleep(30)
except:
    pass
text_classifier = estimator.deploy(initial_instance_count = 1,
                                instance_type = 'ml.t2.medium',
                                endpoint_name=endpoint_name)

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!